In [4]:
from urllib.parse import urlencode
import math
import requests
import time

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 6, Finished, Available, Finished)

In [5]:
url = "https://www.naukri.com/jobapi/v3/search?"

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 7, Finished, Available, Finished)

In [6]:
df_pages = spark.sql("SELECT * FROM bing_lake_db.naukri_pages_to_scrape")
display(df_pages)

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9d0181b2-4e9c-464c-8993-e67ff4a926bc)

In [7]:
# Extract the value (assuming you want the first row, first column)
total_new_jobs_pages = df_pages.collect()[0][0]
total_new_jobs_pages=math.ceil(total_new_jobs_pages/20)

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 9, Finished, Available, Finished)

In [8]:
headers = {
    "appid": "109",
    "systemid": "109",
    "clientid": "d3skt0p",
    "accept": "application/json",
    "accept-encoding": "gzip, deflate, br",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36",
}

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 10, Finished, Available, Finished)

In [10]:
search_location="india"

def seo_keys(input_string):
    
    # Replace spaces with hyphens
    hyphenated_string = input_string.replace(' ', '-')
    
    # Append the static suffix
    static_url = hyphenated_string + f"-jobs-in-{search_location}"
    
    return static_url

payload = {
    "noOfResults": 20,
    "urlType": "search_by_key_loc",
    "searchType": "adv",
    "location": search_location,
    "keyword": job_titles,
    "sort": "r",
    "pageNo": 1,
    "jobAge": 1,
    "seoKey": seo_keys(job_titles),
    "src": "cluster",
}

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 12, Finished, Available, Finished)

In [11]:
def check_string(text, string):
    if text.lower() in string.lower():
        return True
    return False

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 13, Finished, Available, Finished)

In [12]:
job_id=[]
job_title=[]
date_posted=[]
company_name=[]
tech_stack=[]
experience=[]
salary=[]
job_location=[]

StatementMeta(, 27f33b0b-f62d-4564-be0b-00c7cc7ac459, 14, Finished, Available, Finished)

In [ ]:
# Things i'm scrapping [title,footerPlaceholderLabel,companyName,tagsAndSkills,jobDescription]
for page_number in range(1, total_new_jobs_pages):
    payload['pageNo'] = page_number
    search_results = requests.Session().get(url, params=urlencode(payload),  headers=headers).json()
    for i in range(0,19):
        search_results['jobDetails'][i]
        if not check_string(job_titles,search_results['jobDetails'][i]['title']):
            continue
        job_title.append(search_results['jobDetails'][i]['title'])
        job_id.append(search_results['jobDetails'][i]['jobId'])
        date_posted.append(search_results['jobDetails'][i]['footerPlaceholderLabel'])
        company_name.append(search_results['jobDetails'][i]['companyName'])
        tech_stack.append(search_results['jobDetails'][i]['tagsAndSkills'])
        experience.append(search_results['jobDetails'][i]['placeholders'][0]['label'])
        salary.append(search_results['jobDetails'][i]['placeholders'][1]['label'])
        """soup = BeautifulSoup(search_results['jobDetails'][i]['jobDescription'], 'html.parser')
        all_description = []
        description = soup.find('div')
        for des in description.ul.find_all('li'):
                des_text = des.get_text(strip=True)
                all_description.append(des_text)
        print("\n".join(map(str, all_description)))""" # Description Summary will be used in next versions
        job_location.append(search_results['jobDetails'][i]['placeholders'][2]['label'])
    time.sleep(60)  # Wait 1 Minutes between requests

In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
# Create an expected schema
columns = StructType([StructField('job_id', 
                                  StringType(), False),
                    StructField('job_title', 
                                  StringType(), True),
                    StructField('date_posted',
                                StringType(), True),
                    StructField('company_name',
                                StringType(), True),
                    StructField('tech_stack',
                                StringType(), True),
                    StructField('experience',
                                StringType(), True),
                    StructField('salary',
                                StringType(), True),
                    StructField('job_location',
                                StringType(), True)])

In [ ]:
#combine the lists
data = list(zip(job_id,job_title,date_posted,company_name,tech_stack,experience,salary,job_location))

In [ ]:
df=spark.createDataFrame(data,schema=columns)
df.show()

In [ ]:
from pyspark.sql.utils import AnalysisException

try:
    table_name='bing_lake_db.naukri_jobs'
    df.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table Already Exists")
    df.createOrReplaceTempView("vw_df")
    spark.sql(f"""merge into {table_name} target_table
                using vw_df source_view
                on source_view.job_id=target_table.job_id
                when matched and
                target_table.job_id<>source_view.job_id or
                target_table.job_title<>source_view.job_title or
                target_table.date_posted<>source_view.date_posted or
                target_table.company_name<>source_view.company_name or
                target_table.tech_stack<>source_view.tech_stack or
                target_table.experience<>source_view.experience or
                target_table.salary<>source_view.salary or
                target_table.job_location<>source_view.job_location
                then update set *
                when not matched then insert *
                """)